# LSTM (Long Short-Term Memory)

 
### A popular neural network model used to predict future stock prices.


In [1]:
import pandas as pd

file_name='SOL_USDT.csv'
# Load stock price data
data = pd.read_csv(f'../data/raw/{file_name}', parse_dates=True)
data['datetime'] = pd.to_datetime(data['datetime'])

data.set_index('datetime', inplace=True)
data = data[['open', 'high', 'low', 'close', 'volume']]

data

,open,high,low,close,volume
datetime,,,,,
2024-06-15 00:00:00,142.973,143.995,142.854,143.483,242491.00
2024-06-15 01:00:00,143.483,144.466,143.134,143.240,218125.00
2024-06-15 02:00:00,143.240,143.948,143.113,143.693,127771.00
2024-06-15 03:00:00,143.693,144.921,143.677,144.903,217645.00
2024-06-15 04:00:00,144.903,145.172,144.000,144.714,404695.00
...,...,...,...,...,...
2025-06-05 09:00:00,152.000,152.970,151.980,152.950,337017.50
2025-06-05 10:00:00,152.940,152.970,151.350,151.860,656915.88
2025-06-05 11:00:00,151.860,152.350,151.160,152.300,591522.48


### Resample data to daily frequency using OHLC dictionary


In [2]:
ohlc_dict = {                                                                                                             
    'open': 'first',                                                                                                    
    'high': 'max',                                                                                                       
    'low': 'min',                                                                                                        
    'close': 'last',                                                                                                    
    'volume': 'sum',
}

data = data.resample('D', closed='left', label='left').apply(ohlc_dict)
# data = pd.DatetimeIndex(data, freq='D')

data.drop(data.tail(2).index, inplace=True)
data = data.asfreq('D')
data.dropna(inplace=True)


data

,open,high,low,close,volume
datetime,,,,,
2024-06-15,142.973,145.740,142.854,145.450,5510166.00
2024-06-16,145.450,151.347,143.020,151.245,7365593.00
2024-06-17,151.244,151.747,139.765,143.197,14392716.00
2024-06-18,143.197,143.749,127.000,137.359,25248935.00
2024-06-19,137.359,141.876,134.402,135.557,13715932.00
...,...,...,...,...,...
2025-05-30,166.610,167.410,155.010,156.120,25265495.15
2025-05-31,156.110,157.920,152.060,156.350,16603647.07
2025-06-01,156.360,157.850,150.480,157.620,15529303.58


In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# include candlestick with rangeselector
fig.add_trace(go.Candlestick(
                x=data.index,
                open=data['open'], 
                high=data['high'],
                low=data['low'], 
                close=data['close'],
                name='Candle'),
               secondary_y=True)

# include a go.Bar trace for volumes
fig.add_trace(go.Bar(x=data.index, y=data['volume'],marker_color='gray', opacity=0.5,name='Volume'), secondary_y=False)

# Update layout
fig.update_layout(
    title=f'{file_name} Stock High & Low Price', xaxis_title='Date',
    yaxis=dict(title='Volume'),
    yaxis2=dict(title='Price', overlaying='y', side='right'),
    xaxis_rangeslider_visible=False
)

fig.show()

# LSTM (Long Short-Term Memory)

## A popular neural network model used to predict future stock prices.

In [4]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
import plotly.graph_objects as go

from keras.layers import LSTM, Dense, Dropout, Input
from keras.models import Sequential


2025-10-05 12:11:11.069258: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-05 12:11:11.413240: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-05 12:11:12.506282: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
# split into train and test sets with the ratio of 0.8 
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size
train_data, test_data = data.iloc[0:train_size], data.iloc[train_size:len(data)]


# Scaling



## Deep learning (LSTM/GRU for time-series):
1. 👉 MinMaxScaler(feature_range=(-1, 1)) or StandardScaler are most common.
2. MinMaxScaler helps if you want bounded activations (e.g., sigmoid/tanh).
3. StandardScaler helps if you expect out-of-range values (safer for real-world market data).

### Should all OHLC features share the same scale?

* Yes ✅ for open, high, low, close (OHLC) you should use the same scaler:
* They are in the same units (price).
* If you scale them independently, you destroy relative relationships (e.g., “close > open” might be lost after scaling).
* Using a shared scaler keeps the ratios and differences consistent.

### What about volume?

* Volume is on a completely different scale (can be 1e3 vs. 1e8 for different stocks).
* 👉 Best practice: scale volume separately (fit another scaler just for volume).
* This prevents it from distorting the OHLC scaling.
* You can even log-transform volume first (log(1+volume)) to compress heavy-tailed spikes, then scale.

In [6]:
ohlc_scaler = MinMaxScaler(feature_range=(0, 1))
ohlc_scaled_data = ohlc_scaler.fit_transform(train_data[['open', 'high', 'low', 'close']])
volume_scaler = MinMaxScaler(feature_range=(0, 1))
volume_scaled_data = volume_scaler.fit_transform(train_data[['volume']])

scaled_train_data = np.hstack((ohlc_scaled_data, volume_scaled_data))

In [7]:
def data_prep(data, sequence_length=5):
    
    # Prepare data for LSTM
    x_train, y_train = [], []
    for i in range(sequence_length, len(data)):
        x_train.append(data[i-sequence_length:i, :])
        y_train.append(data[i, [1, 2]])  # Predict high and low prices

    x_train, y_train = np.array(x_train), np.array(y_train)
    return x_train, y_train

In [8]:

x_train, y_train = data_prep(scaled_train_data, sequence_length=5)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))

# %%
# Build LSTM model
model = Sequential()
model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25, activation='relu'))
model.add(Dense(units=2))  # Predict high and low prices

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Fit the model
model.fit(x_train, y_train, epochs=50, batch_size=5)


I0000 00:00:1759646473.223265   10749 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9507 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/50


2025-10-05 12:11:15.664861: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91301


56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0644
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0132
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0129
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0126
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0120
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0117
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0096
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0096 
Epoch 9/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0107
Epoch 10/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0085
Epoch 11/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0086
Epoch 12/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0103
Epoch 13/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0092
Epoch 14/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0097
Epoch 15/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0082
Epoch 16/50
56/56 ━━━━━━━━━━━

In [9]:
ohlc_scaled_data = ohlc_scaler.transform(test_data[['open', 'high', 'low', 'close']])
volume_scaled_data = volume_scaler.transform(test_data[['volume']])
scaled_test_data = np.hstack((ohlc_scaled_data, volume_scaled_data))
x_test, y_test = data_prep(scaled_test_data, sequence_length=5)


# Forecast future prices
forecast_length = len(y_test)
# last_x = x_test[-1]
forecast_prices = []

for i in range(forecast_length):
    test_row = x_test[i]
    # Predict high and low prices
    scaled_pred = model.predict(np.array([test_row]))  # shape (1, 2)
    scaled_high, scaled_low = scaled_pred[0]
    # create a dummy row for inverse transform
    dummy = np.zeros((1, 4))  # because ohlc_scaler was fit on (open, high, low, close)
    dummy[0, 1] = scaled_high  # place high at index 1
    dummy[0, 2] = scaled_low   # place low at index 2
    # inverse transform
    inversed = ohlc_scaler.inverse_transform(dummy)
    high_price, low_price = inversed[0, 1], inversed[0, 2]
    print(f"Forecasted High: {high_price}, Low: {low_price}")
    # append to results
    forecast_prices.append([high_price, low_price])


forecast_prices = np.array(forecast_prices)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
Forecasted High: 136.2129153445363, Low: 125.07258357337118
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Forecasted High: 134.97106252647936, Low: 124.47552083289624
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Forecasted High: 134.54220809452235, Low: 124.34308905208111
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Forecasted High: 135.29724191352724, Low: 125.02025400814415
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Forecasted High: 133.4632300067693, Low: 123.4773989264071
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Forecasted High: 131.96460814625024, Low: 121.39104065613449
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Forecasted High: 132.03789656460285, Low: 121.3825131764561
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Forecasted High: 133.0112737365067, Low: 122.6283419891894
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Forecasted High: 130.6193314999342, Low: 120.7570408473313
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Forecasted High: 127.567507712543, Low: 119.8288280028403
1/1 ━━━━━━━━━━━━━━━━

In [10]:
test_data.loc[test_data.index[-forecast_length:], 'forecast_high'] = forecast_prices[:,0]
test_data.loc[test_data.index[-forecast_length:], 'forecast_low'] = forecast_prices[:,1]
forecast_df = test_data


/tmp/ipykernel_10749/3415770437.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_10749/3415770437.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
# Create Plotly graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['high'], name='Actual High Price'))
fig.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['forecast_high'], name='Forecast High Price'))
fig.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['low'], name='Actual Low Price'))
fig.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['forecast_low'], name='Forecast Low Price'))
fig.update_layout(title=f'{file_name} Stock Price Prediction using LSTM', xaxis_title='Date', yaxis_title='Price')
fig.show()



In [12]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['forecast_high'], name='Forecast High Price'))
fig.add_trace(go.Scatter(x=forecast_df.index, y=forecast_df['forecast_low'], name='Forecast Low Price'))

fig.add_trace(go.Candlestick(
                x=data.index,
                open=data['open'], 
                high=data['high'],
                low=data['low'], 
                close=data['close'],
                name='Candle'),)

fig.update_layout(title=f'{file_name} Stock Price Prediction using LSTM', xaxis_title='Date', yaxis_title='Price',    xaxis_rangeslider_visible=False )
fig.show()


In [13]:
forecast_df.tail(20)


,open,high,low,close,volume,forecast_high,forecast_low
datetime,,,,,,,
2025-05-15,176.51,178.23,166.55,169.06,25491932.66,185.927510,172.206246
2025-05-16,169.06,174.16,166.22,167.33,18061064.71,177.025484,163.927457
2025-05-17,167.33,169.95,163.91,165.87,16744002.36,175.009630,162.049292
2025-05-18,165.87,176.73,164.50,173.24,22124057.51,172.488736,160.007064
2025-05-19,173.25,173.87,159.28,166.76,27244529.53,177.819284,165.088967
2025-05-20,166.76,173.06,164.40,168.51,21947126.47,172.420511,160.117611
2025-05-21,168.51,175.92,165.34,173.50,33065481.41,174.979054,162.472502
2025-05-22,173.50,180.82,172.40,179.60,23205120.54,178.105045,165.162197
2025-05-23,179.61,187.67,173.18,173.92,37351156.03,185.949444,172.177968
